<a href="https://colab.research.google.com/github/dasom222g/learn-LLM/blob/main/02_10_n_gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# n-gram 이해하기
- 2개 이상의 토큰을 하나의 토큰으로 묶어서 토큰 확장

## konlpy 설치

In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 26.9 MB/s eta 0:00:00


In [ ]:
# 이미 앞서 다뤄본 적이 있는 기본적인 라이브러리
import numpy as np
import pandas as pd

import json
import re

from tqdm.notebook import tqdm

In [ ]:
from konlpy.tag import Okt # komoran, hannanum, kkma, mecab

In [ ]:
import os
from datetime import datetime

## Download Dataset(github에서 NSMC 데이터셋 다운로드)

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', header=0, delimiter='\t' ,quoting=3)
test = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', header=0, delimiter='\t' ,quoting=3)

In [ ]:
train.shape, test.shape

((150000, 3), (50000, 3))

In [ ]:
display(train.head())
display(test.head())

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
train.isnull().sum()

,0
id,0
document,5
label,0


In [ ]:
test.isnull().sum()

,0
id,0
document,3
label,0


In [ ]:
train.dropna(inplace=True)
test.dropna(inplace=True)

In [ ]:
X_train = train.document
y_train = train.label

X_test = test.document
y_test = test.label
X_train[0], y_train[0]

('아 더빙.. 진짜 짜증나네요 목소리', np.int64(0))

In [ ]:
print('#Train set size:', len(X_train))
print('#Test set size:', len(X_test))

#Train set size: 149995
#Test set size: 49997


### N-gram (bi-gram)

In [ ]:
review = X_train[0]
review

'아 더빙.. 진짜 짜증나네요 목소리'

In [ ]:
okt = Okt()

f = open('nsmc_stopwords.txt')
stop_words = f.read().split()

# 1. 한글 및 공백을 제외한 문자 모두 제거.
review_text = re.sub("[^가-힣\\s]", "", review)
# 2. okt 객체를 활용해서 형태소 토큰화 + 품사 태깅
word_review = okt.pos(review_text, stem=True)

# 노이즈 & 불용어 제거
word_review = [(token, pos) for token, pos in word_review if not token in stop_words and len(token) > 1]

# 명사, 동사, 형용사 추출
word_review = [token for token, pos in word_review if pos in ['Noun', 'Verb', 'Adjective']]
print(word_review)

['더빙', '진짜', '짜증나다', '목소리']


In [ ]:
def bigram(word_review):
    return [' '.join(word_review[i:i+2]) for i in range(len(word_review)-1)]

In [ ]:
bigram(word_review)

['더빙 진짜', '진짜 짜증나다', '짜증나다 목소리']

In [ ]:
word_review += bigram(word_review)
word_review

['더빙', '진짜', '짜증나다', '목소리', '더빙 진짜', '진짜 짜증나다', '짜증나다 목소리']

### 종합적인 구현

In [73]:
def preprocessing(review, stop_words):
    # 기존 전처리에 n-gram 추가 적용하기
    okt = Okt()

    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    review_text = re.sub("[^가-힣\\s]", "", review)

    # 2. okt 객체를 활용해서 형태소 토큰화 + 품사 태깅
    word_review = okt.pos(review_text)

    # 노이즈 & 불용어 제거
    word_review = [(word, pos) for word, pos in word_review if word not in stop_words and len(word) > 1]

    # 명사, 동사, 형용사 추출
    word_review = [word for word, pos in word_review if pos in ['Noun', 'Verb', 'Adjective']]
    print(f"n-gram 전: {word_review}")

    # n-gram : bi-gram 적용해서 기존 토큰 뭉치에 합치기
    bi_gram_review = [word + ' ' + word_review[idx + 1] for idx, word in enumerate(word_review) if idx < len(word_review) - 1]
    print(f"n-gram 후: {bi_gram_review}")

    return word_review + bi_gram_review

In [74]:
f = open('nsmc_stopwords.txt')
stop_words = f.read().split()

reviews = []

for review in tqdm(X_train[:1000]):
    reviews.append(preprocessing(review, stop_words))

  0%|          | 0/1000 [00:00<?, ?it/s]

n-gram 전: ['더빙', '진짜', '짜증나네요', '목소리']
n-gram 후: ['더빙 진짜', '진짜 짜증나네요', '짜증나네요 목소리']
n-gram 전: ['포스터', '보고', '초딩', '오버', '연기', '가볍지', '않구나']
n-gram 후: ['포스터 보고', '보고 초딩', '초딩 오버', '오버 연기', '연기 가볍지', '가볍지 않구나']
n-gram 전: ['무재', '밓었', '다그', '래서', '보는것을', '추천']
n-gram 후: ['무재 밓었', '밓었 다그', '다그 래서', '래서 보는것을', '보는것을 추천']
n-gram 전: ['교도소', '이야기', '구먼', '솔직히', '재미', '없다', '평점', '조정']
n-gram 후: ['교도소 이야기', '이야기 구먼', '구먼 솔직히', '솔직히 재미', '재미 없다', '없다 평점', '평점 조정']
n-gram 전: ['몬페', '익살스런', '연기', '돋보였던', '스파이더맨', '늙어', '보이기만', '했던', '커스틴', '던스트', '이뻐', '보였다']
n-gram 후: ['몬페 익살스런', '익살스런 연기', '연기 돋보였던', '돋보였던 스파이더맨', '스파이더맨 늙어', '늙어 보이기만', '보이기만 했던', '했던 커스틴', '커스틴 던스트', '던스트 이뻐', '이뻐 보였다']
n-gram 전: ['걸음', '초등학교', '학년', '생인', '살용', '반개', '아까']
n-gram 후: ['걸음 초등학교', '초등학교 학년', '학년 생인', '생인 살용', '살용 반개', '반개 아까']
n-gram 전: ['원작', '긴장감', '제대로', '살려내지못', '했다']
n-gram 후: ['원작 긴장감', '긴장감 제대로', '제대로 살려내지못', '살려내지못 했다']
n-gram 전: ['반개', '아깝다', '나온다', '이응경', '길용우', '생활', '정말', '해도', '그것', '낫겟다', '납치', '감금'

In [78]:
reviews[:100]
for reviews_datas in reviews:
  if len(reviews_datas) <= 2:
    print(reviews_datas)


['재밋는뎅']
['재미있어요']
['좋아요']
['최고']
['심심한']
['졸작']
['킬링타임']
['대박']
['별로']
['빵점']
['좋구나']
['만해']
[]
['좋은']
[]
['로큰롤']
['알바']
['버려']
['최고']
['감동']
['훈훈하네요']
['미로']
['연기']
['재미있']
['좋은']
['웃겨']
['망함']
[]
['슬픔']
['좋아']
['처주']
['주냐']
['신선하네']
['안습']
['없나']
['재밌는데']
['솔직히']
[]
['대강']
['슬프다']
[]
['한국']
[]
['최고']
[]
['봤습니다']
['감동']
[]
[]
[]
['감동']
['만들기']
[]
['망작']
[]
['괜찮은']
[]
[]
['재밌는']
['쓰레기']
['없다']
[]
[]
['더빙']
['재밌다']
['있었다']
['아깝다']
[]
['잼있던데']
['좋은']
['빠다']
['재미없음']
['없네요']
[]
['좋은']
['귀여운']
['전형']
['믿으려는']
[]
['재밌군']
['재밌는데']
['좋았다']
['재밌']
['개안음']
['예원']
['없나요']
[]
['재밌었어요']
['재미없음']


In [ ]:
# @title 정답
def preprocessing(review):
    # 기존 전처리에 n-gram 추가 적용하기
    okt = Okt()

    f = open('nsmc_stopwords.txt')
    stop_words = f.read().split()

    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    review_text = re.sub("[^가-힣\\s]", "", review)

    # 2. okt 객체를 활용해서 형태소 토큰화 + 품사 태깅
    word_review = okt.pos(review_text, stem=True)

    # 노이즈 & 불용어 제거
    word_review = [(token, pos) for token, pos in word_review if not token in stop_words and len(token) > 1]

    # 명사, 동사, 형용사 추출
    word_review = [token for token, pos in word_review if pos in ['Noun', 'Verb', 'Adjective']]
    # print(word_review)

    # n-gram : bi-gram
    n_gram = [' '.join(word_review[i:i+2]) for i in range(len(word_review)-1)]
    # print('n-gram')
    # print(n_gram)

    # 기존 토큰 뭉치 + n-gram
    word_review += n_gram
    # print('n-gram 토큰 추가된 뭉치')
    # print(word_review)

    return word_review

In [ ]:
reviews = []

for review in tqdm(X_train[:1000]):
    reviews.append(preprocessing(review))

  0%|          | 0/1000 [00:00<?, ?it/s]

['더빙', '진짜', '짜증나다', '목소리']
n-gram
['더빙 진짜', '진짜 짜증나다', '짜증나다 목소리']
n-gram 토큰 추가된 뭉치
['더빙', '진짜', '짜증나다', '목소리', '더빙 진짜', '진짜 짜증나다', '짜증나다 목소리']
['포스터', '보고', '초딩', '오버', '연기', '가볍다', '않다']
n-gram
['포스터 보고', '보고 초딩', '초딩 오버', '오버 연기', '연기 가볍다', '가볍다 않다']
n-gram 토큰 추가된 뭉치
['포스터', '보고', '초딩', '오버', '연기', '가볍다', '않다', '포스터 보고', '보고 초딩', '초딩 오버', '오버 연기', '연기 가볍다', '가볍다 않다']
['무재', '밓었', '다그', '래서', '보다', '추천']
n-gram
['무재 밓었', '밓었 다그', '다그 래서', '래서 보다', '보다 추천']
n-gram 토큰 추가된 뭉치
['무재', '밓었', '다그', '래서', '보다', '추천', '무재 밓었', '밓었 다그', '다그 래서', '래서 보다', '보다 추천']
['이야기', '구먼', '솔직하다', '재미', '없다', '조정']
n-gram
['이야기 구먼', '구먼 솔직하다', '솔직하다 재미', '재미 없다', '없다 조정']
n-gram 토큰 추가된 뭉치
['이야기', '구먼', '솔직하다', '재미', '없다', '조정', '이야기 구먼', '구먼 솔직하다', '솔직하다 재미', '재미 없다', '없다 조정']
['몬페', '익살스럽다', '연기', '돋보이다', '늙다', '보이다', '커스틴', '던스트', '이쁘다', '보이다']
n-gram
['몬페 익살스럽다', '익살스럽다 연기', '연기 돋보이다', '돋보이다 늙다', '늙다 보이다', '보이다 커스틴', '커스틴 던스트', '던스트 이쁘다', '이쁘다 보이다']
n-gram 토큰 추가된 뭉치
['몬페', '익살스럽다', '연기', '돋보이다', '늙다', 

### 벡터화
1. BoW : CountVectorizer
2. TF-IDF : TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

bow = CountVectorizer(tokenizer=preprocessing, min_df=5, max_df=0.5)

# X_train_bow = bow.fit(X_train)
# X_train_bow = bow.fit_transform(X_train)
# X_test_bow = bow.transform(X_test)

In [ ]:
tfidf = TfidfVectorizer(tokenizer=preprocessing, max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

X_train_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#### n-gram 파라미터 활용하기

ngram_range : tuple (min_n, max_n), default=(1, 1)  
(1, 1) means only unigrams,  
(1, 2) means unigrams and bigrams,  
(2, 2) means only bigrams.

In [ ]:
tfidf = TfidfVectorizer(tokenizer=preprocessing, ngram_range=(1,2), max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

X_train_tfidf.toarray()

## 모델 학습 및 검증

In [ ]:
# 모델 1 : Logistic Regression 모형
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression()
log_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(log_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(log_clf.score(X_test_tfidf, y_test)))

Train set score: 0.816
Test set score: 0.809


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
log_clf = LogisticRegression()
log_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(log_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(log_clf.score(X_test_tfidf, y_test)))

Train set score: 0.818
Test set score: 0.810
